## Lane segmentation
This notebooks contains the code we used to train image segmentation to detect the lane markers from an RGB image

### 1. Prepare data
From carla we obtained RGB images and their respective segmentation image. We need to convert those images into a binary mask and create a dataset for them. To this end we require the following structure before hand:
- data/
    - raw/
        - rgb_images/
        - segmentation_images/


In [2]:

%load_ext autoreload
import os
import sys

root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if root_path not in sys.path:
    sys.path.append(root_path)


In [3]:
import re
import cv2
import numpy as np
import lightning as L
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from tqdm import tqdm
from loguru import logger
import wandb
%autoreload 2
from DAI.cv.traffic_lane_segmentation import TrafficLaneSegmentationDataModule, TrafficLaneSegmentationModel
logger.remove()
logger.add(lambda msg: tqdm.write(msg, end=''))

1

In [4]:
data_root = os.path.abspath("../data") ## This should point to the data directory as described above.
print(data_root)
os.makedirs(os.path.join(data_root, 'images'), exist_ok=True)
os.makedirs(os.path.join(data_root, 'labels'), exist_ok=True)

d:\Soja\1_Masters\AI Lab\AI project\Lane detect integration\DAI_project\data


In [8]:
traffic_lane_color = np.array(([157, 234, 50]))

image_set_length = len(os.listdir(os.path.join(data_root, 'raw', 'rgb_images')))
for image_name in tqdm(os.listdir(os.path.join(data_root, 'raw', 'rgb_images'))):
    image_number = re.findall('image(\d+).png', image_name)[0]
    rgb_image_path = os.path.join(data_root, 'raw', 'rgb_images', image_name)
    seg_image_path = os.path.join(data_root, 'raw', 'segmentation_images',f'seg{image_number}.png')
    try:
        rgb_image = cv2.imread(rgb_image_path)
        seg_image = cv2.imread(seg_image_path)
    except Exception: 
        continue
    
    if rgb_image is None or seg_image is None:
        logger.warning(f"skipping {rgb_image_path}")
        continue
    mask: np.ndarray = np.any(seg_image == traffic_lane_color, axis=-1)
    binary_segmentation_image = (mask*255).astype(np.uint8)
    cv2.imwrite(os.path.join(data_root, 'images', image_name), rgb_image)
    cv2.imwrite(os.path.join(data_root, 'labels', f"segm{image_number}.png"), binary_segmentation_image)


100%|██████████| 881/881 [01:51<00:00,  7.88it/s]


### 2 Prepare Model
We use the torchvision DeepLabV3 segmentation model which we wrapped with a lightning module

In [ ]:

data_loader = TrafficLaneSegmentationDataModule(data_root, batch_size=2)
model = TrafficLaneSegmentationModel()
wandb.finish()

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇██
train_loss,█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▂▂▂▁▁▂▂▂▂▂▂▁▂▂▁▂▁▁▂▁
trainer/global_step,▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇███
val_loss,▁█▂▁▁▁▁▁▁▅▁▁▁▁
epoch,13
train_loss,0.00052
trainer/global_step,4815
val_loss,0.00411


### 3 Train model
We setup training with Weights and biases early stopping and model checkpointing

In [ ]:
save_callback = ModelCheckpoint('./models', 'best.pt', monitor='val_loss')
early_stopper = EarlyStopping(monitor='val_loss', patience=5)
logging = WandbLogger(project='Traffic-lane-segmentation', name='lraspp_mobilenet_v3_large')
trainer = L.Trainer(accelerator='auto', callbacks=[save_callback, early_stopper], logger=logging)
trainer.fit(model, datamodule=data_loader, )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
d:\Soja\1_Masters\AI Lab\AI project\Lane detect integration\DAI_project\.venv\lib\site-packages\lightning\pytorch\loops\utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.


d:\Soja\1_Masters\AI Lab\AI project\Lane detect integration\DAI_project\.venv\lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:652: Checkpoint directory D:\Soja\1_Masters\AI Lab\AI project\Lane detect integration\DAI_project\notebooks\models exists and is not empty.

  | Name  | Type              | Params | Mode 
----------------------------------------------------
0 | model | DeepLabV3         | 11.0 M | train
1 | loss  | BCEWithLogitsLoss | 0      | train
----------------------------------------------------
11.0 M    Trainable params
0         Non-trainable params
11.0 M    Total params
44.081    Total estimated model params size (MB)


Epoch 13: 100%|██████████| 344/344 [01:31<00:00,  3.75it/s, v_num=4ca4]    
